In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QDockWidget, QTextEdit
from PyQt5.QtCore import Qt, QTimer
import filepath
import pdb
from panda3d.core import (
    loadPrcFileData
)
from qpanda3d import QShowBase, QPanda3DWidget, QControl, Synchronizer
from demos.physics_room import RoomScene
from panda3d_game.app import ControlShowBase, ContextShowBase, PhysicsShowBase
from vispyutil.canvas import SynchronizedCanvas
from vispyutil.showbase import CanvasBackgroundShowBase
import numpy as np
from vispy import app, scene, visuals,use
from sympy.physics.units import (
    kilometer, meter,centimeter,
    gram, kilogram, tonne,
    newton, second
)
from util.physics import autocomplete_units, G_val, getG
from demos.ball_room import MassedBall,tmoon

In [2]:
from PyQt5.QtGui import QPalette, QColor
from typing import Tuple
def hex2pal(hexstr:str) -> QColor:
    hexstr = hexstr.lstrip('#')
    return QColor(
        int(hexstr[:2],16),
        int(hexstr[2:4],16),
        int(hexstr[4:6],16)
    )
def hex2rgb(hexstr:str) -> Tuple:
    hexstr = hexstr.lstrip('#')
    return (
        int(hexstr[:2],16),
        int(hexstr[2:4],16),
        int(hexstr[4:6],16)
    )

def hex2rgbastr(hexstr,a=1) -> str:
    rgb_tup = hex2rgb(hexstr)
    return "rgba({},{},{},{})".format(
        rgb_tup[0],
        rgb_tup[1],
        rgb_tup[2],
        a
    )

# c_violet_frame = hex2pal('#97739e')
# c_violet_bg = hex2pal('#472c4d')
# c_gold_text = hex2pal('#472c4d')
# c_orange_text = hex2pal('#f5a16d')
c_violet_frame = '#97739e'
c_violet_bg = '#472c4d'
c_orange_text = '#f5a16d'
styleSheet = (f"""
/*QDockWidget [
    border: 2px solid black;
    background-color: lightblue;
]*/
QMainWindow [
    background-color: {hex2rgbastr(c_violet_frame)};
]
QTextEdit, QListView [
    background-color: {hex2rgbastr(c_violet_bg)};
    color: {hex2rgbastr(c_orange_text)};
    font-size: 20px;
    font-family: 'Courier'; /* fixme */
    /* background-image: url(draft.png); */
    /* background-attachment: fixed; */
]
""").replace('[','{').replace(']','}')

In [3]:

# palette = QPalette()
# palette.setColor(


class StarCanvas(SynchronizedCanvas):
    def __init__(self, n_stars, rho=10):
        SynchronizedCanvas.__init__(self)
        self.n_stars = n_stars 
        # randomly pick stars
        np.random.seed(0)  # todo: use torch random state
        cos_pos_theta = np.random.uniform(-1,1, n_stars)
        pos_theta = np.arccos(cos_pos_theta)
        pos_phi = np.random.uniform(0,2*np.pi, n_stars)
        sizes = np.random.uniform(0, 5, n_stars) 
        r = rho * np.sin(pos_theta)
        x = r * np.cos(pos_phi)
        y = r * np.sin(pos_phi)
        z = rho * np.cos(pos_theta)
        positions = np.vstack([x,y,z]).T
        scatter = scene.visuals.Markers()
        scatter.set_data(positions, edge_color=None, face_color='white', size=sizes, symbol='o')
        # Add scatter plot to the view
        self.view.add(scatter)


class StarScene(CanvasBackgroundShowBase):
    def __init__(self):
        # ControlShowBase.__init__(self)
        self.stars_canvas = StarCanvas(60000)
        CanvasBackgroundShowBase.__init__(self,self.stars_canvas)

        
class PlanetStarScene(StarScene,PhysicsShowBase):
    def __init__(self):
        StarScene.__init__(self)
        PhysicsShowBase.__init__(self)
        self.bullet_world.setGravity((0,0,0))
        self.unit = {
            "mass" : tonne,
            "length" : 100*meter,
            "time": 1 * second,
            # "force" : sp.Number(1e3) * newton
        }
        autocomplete_units(self.unit)
        self.planet1 = MassedBall(
            name="planet1",
            radius=1000*meter,
            mass=1e6*tonne,
            units=self.unit
        )
        self.planet1.reparentTo(self.render)
        self.planet1.set_texture(tmoon)
        self.planet1.setScale(10)
        self.planet1.setPos(0,10,0)

class PlanetStarSpace(PlanetStarScene, QControl):
    def __init__(self, isQt = True):
        import pdb
        # pdb.set_trace()
        QControl.__init__(self)
        PlanetStarScene.__init__(self)
        self.isQt = isQt
        if self.isQt:
            self.startQt()

class ConsoleWidget:
    pass # set a console for it

class TextBuffer:
    # FIXME: use a queue instead
    # implement: write to a cache file
    # cached queue
    pass

class TextBrowserWidget:
    # browse a buffer
    pass


class MainWindow(QMainWindow):
    def __init__(self, FPS=60, stylesheet=styleSheet):
        super().__init__()

        # central_widget = QTextEdit("Central Widget")
        # self.setCentralWidget(central_widget)
        self.FPS = FPS

        # Create three dock widgets
        self.dock_top_left = QDockWidget("Game Camera", self)
        self.dock_bottom_left = QDockWidget("Console", self)
        self.dock_right = QDockWidget("Logger", self)
        
        # Add the docks to the main window
        self.addDockWidget(Qt.LeftDockWidgetArea, self.dock_top_left)
        self.addDockWidget(Qt.RightDockWidgetArea, self.dock_right)
        
        # Split the left dock area vertically (top and bottom)
        self.splitDockWidget(self.dock_top_left, self.dock_bottom_left, Qt.Vertical)

        # You can adjust the sizes of the docks
        self.resizeDocks([self.dock_top_left, self.dock_bottom_left], [200, 200], Qt.Vertical)

        self.setWindowTitle("Three Dock Layout")
        self.resize(800, 600)

        # self.panda3d = TestRoom()
        self.panda3d = None
        self.synchronizer = Synchronizer(self.FPS)
        loadPrcFileData("", "window-type offscreen")
        self.startGame()
        # TODO: destroy
        # Set widget content for the docks (could be any widget)
        # self.dock_top_left.setWidget(QTextEdit("Top Left Content"))
        
        self.dock_bottom_left.setWidget(QTextEdit("Bottom Left Content"))
        self.dock_right.setWidget(QTextEdit("Right Content"))
        # botleftpal = self.dock_bottom_left.palette()
        # botleftpal.setColor(QPalette.Window, QColor('blue'))
        # self.dock_bottom_left.setPalette(botleftpal)
        self.setStyleSheet(styleSheet)
        if stylesheet is not None:
            self.dock_bottom_left.setStyleSheet(stylesheet)
        
        # ToDO; set loggers
        
        # pdb.set_trace()

    def startGame(self):
        self.panda3d = PlanetStarSpace()
        self.synchronizer.setShowBase(self.panda3d)
        self.pandaWidget = QPanda3DWidget(
            self.panda3d, 
            synchronizer=self.synchronizer
        )
        # TODO: log, std
        # print(self.panda3d.parent)
        self.synchronizer.addWidget(self.pandaWidget)
        self.dock_top_left.setWidget(self.pandaWidget)
        self.synchronizer.start()
        self.panda_mouse_watcher = self.panda3d.mouseWatcherNode # FIXME: let a widget watch this
        self.pandaWidget.setFocus()
        # self.panda3d.cam_sensitivity = 0.01

    # todo: remove a widget

    

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


init ContextShowBase
---set ref---:render,<class 'panda3d.core.NodePath'>


/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1711322746054/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SystemExit: 0

/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
